In [1]:
import numpy as np
import pandas as pd
from support import query_data
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
import sqlite3
from support import query_data
%matplotlib inline

In [2]:
db = sqlite3.connect("/data/amazon-fine-foods/amazon-fine-foods/database.sqlite")

In [3]:
reviews, rev_count, user_count = query_data(75, db)

# Results
With the three models previously tested, RandomForestClassifier produces the best results. GridSearchCV was heavily used to tweak the classifier to yield the greatest results. Those results are shown below.

In [4]:
PUNCTUATION = '`~!@#$%^&*()_-+={[}]|\:;"<,>.?/}\t\n'
def process(x):
    """
    Basic Preprocessor to remove punctuation from words. 
    """
    return x.strip(PUNCTUATION)

In [5]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(preprocessor=process, norm='l1', ngram_range=(1, 2), max_df=1.0, max_features=18000)),
    ('rf', RandomForestClassifier(n_estimators=500)),
])

In [6]:
X = reviews["Text"]
y = reviews["UserId"]

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

Fit the data can take about a minute. I believe this is due to the number of estimators in the random forest.

In [8]:
model = pipeline.fit(X_train, y_train)

In [9]:
y_pred = model.predict(X_test)

In [10]:
accuracy_score(y_test, y_pred)

0.75975794251134643

After the numerous model iterations have completed we end up with the following model. The accuracy level is suprisingly high for a datasubset of 100 different users and I am satsfied with the results. 